In [1]:
import re
from ast import literal_eval
import numpy as np
import pandas as pd
from pprint import pprint
import collections
import math
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
#scipy
import scipy 
from sklearn.cluster import KMeans

from urduhack.preprocess import remove_punctuation
from urduhack import stop_words
from urduhack import tokenization as tok
from urduhack import preprocess
from urduhack import utils
from urduhack import normalization as norm
from utilities import words as urduwords
from urduhack import stop_words
from sklearn import metrics
import os
from kneed import KneeLocator
from sklearn.datasets import fetch_20newsgroups
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import CRFTagger
# NLTK Stop words
# Extract noun chunks from corpus
import math


stopwords = list(stop_words.STOP_WORDS)

In [3]:
from SNgramExtractor import SNgramExtractor
import stanza
from spacy_stanza import StanzaLanguage


In [4]:
snlp = stanza.Pipeline(lang="ur")
nlp = StanzaLanguage(snlp)


2020-09-17 14:32:22 INFO: Loading these models for language: ur (Urdu):
| Processor | Package |
-----------------------
| tokenize  | udtb    |
| pos       | udtb    |
| lemma     | udtb    |
| depparse  | udtb    |

2020-09-17 14:32:22 INFO: Use device: cpu
2020-09-17 14:32:22 INFO: Loading: tokenize
2020-09-17 14:32:22 INFO: Loading: pos
2020-09-17 14:32:23 INFO: Loading: lemma
2020-09-17 14:32:23 INFO: Loading: depparse
2020-09-17 14:32:25 INFO: Done loading processors!


In [5]:
dataset = pd.read_csv('urduNewsHeadlines-clusteringDS.csv',encoding='utf-8',sep='\t')

In [6]:
dataset.shape

(260, 6)

In [7]:
dataset

,DateTime,Source,Title,lang,search_keywords,Label
0,2019-09-20 00:54:51,UrduPoint,سی پیک منصوبہ ختم،آئی ایم ایف کی شرائط پرعملدر...,ur,1,CPEC
1,2019-09-20 05:45:20,UrduPoint,حکومت نے سی پیک کے تحت اکانامک زونز پر توجہ مر...,ur,1,CPEC
2,2019-09-20 18:53:21,UrduPoint,سی پیک کاموجودہ مرحلہ پاکستانی عوام کیلئے معاش...,ur,1,CPEC
3,2019-09-21 09:28:12,UrduPoint,وزیر اعلیٰ خیبر پختونخوا محمود خان کا جمعہ کے ...,ur,1,CPEC
4,2019-09-23 06:40:21,UrduPoint,غربت کی شرح میں کمی سی پیک دوسرے مرحلے کی اہم...,ur,1,CPEC
5,2019-09-23 06:40:21,UrduPoint,حکومت عوام کو ریلیف کی فراہمی کے لئے پر عزم ہے...,ur,1,CPEC
6,2019-09-24 06:20:17,UrduPoint,چین پاکستان اقتصادی راہداری منصوبہ کے دوسرے مر...,ur,1,CPEC
7,2019-10-07 11:01:26,UrduPoint,وزیر اعظم عمران خان کے دورہ چین سے دونوں ممالک...,ur,1,CPEC
8,2019-10-07 11:08:21,ExpressNews,صدارتی آرڈیننس کے ذریعے سی پیک اتھارٹی قائم کر...,ur,1,CPEC
9,2019-10-07 15:09:59,UrduPoint,خیبرپختونخوا حکومت کی تمام متعلقہ محکموں کو تر...,ur,1,CPEC


In [8]:
not_stops = ['سی','درمیان','شان','حصہ','خلاف','ضرورت','دسترس','ضروری','دلچسپی']

In [9]:
new_stops = [x for x in stopwords if x not in not_stops]

In [10]:
def purity_score(y_true, y_pred):
    # compute contingency matrix (also called confusion matrix)
    contingency_matrix = metrics.cluster.contingency_matrix(y_true, y_pred)
    # return purity
    return np.sum(np.amax(contingency_matrix, axis=0)) / np.sum(contingency_matrix) 

# Tokenize and lemmatize
def preprocess_func(text):
    text = text.replace('\n'," ")
    text = text.replace('\r'," ")
    text = preprocess.normalize_whitespace(text)
    text = preprocess.remove_punctuation(text)
    text = norm.remove_diacritics(text)
    text = urduwords.fix_join_words(text)
    text = re.sub(r"http\S+", "", text)
    text = re.sub('@[^\s]+','',text)
    nonalpha = re.compile(r"[a-zA-Z0-9.@#_:)(-]")
    text = re.sub(nonalpha,'',text).strip()#remove english alphabets
    wordsC = text.split()
    non_stops = [w for w in wordsC if w not in stopwords]
    cleanText = " ".join(non_stops)
    return cleanText 
        
def candidate_label_lemmatized(label):
    w_labels = label.split()
    lemmatized = lemmatization(w_labels)
    return " ".join(lemmatized)

def news_group_target_names(k):
    return newsgroups_train.target_names[k]
def return_preprocessed_text(words):
    x = literal_eval(words)
    return " ".join(x)
def get_pd_of_cluster(dataset,cluster_idx):
    dataset_sub = dataset[dataset['km_label'] == cluster_idx]
    cnt = CountVectorizer()
    voc = cnt.fit_transform(dataset_sub.preprocessed_text)
    vocab = list(cnt.get_feature_names())
    counts = voc.sum(axis=0).A1
    freq_distribution = Counter(dict(zip(vocab, counts)))
    prob = {}
    for w in vocab:
        prob[w] = freq_distribution[w]/np.sum(counts)
    return prob
def get_words_from_tags(postags):
    words = []
    tags = []
    for u,v in postags:
        words.append(u)
        tags.append(v)
    return words,tags
def pos_regex_matches(doc, pattern,tgs):
    """
    Extract sequences of consecutive tokens from a spacy-parsed doc whose
    part-of-speech tags match the specified regex pattern.

    Args:
        doc (``textacy.Doc`` or ``spacy.Doc`` or ``spacy.Span``)
        pattern (str): Pattern of consecutive POS tags whose corresponding words
            are to be extracted, inspired by the regex patterns used in NLTK's
            `nltk.chunk.regexp`. Tags are uppercase, from the universal tag set;
            delimited by < and >, which are basically converted to parentheses
            with spaces as needed to correctly extract matching word sequences;
            white space in the input doesn't matter.

            Examples (see ``constants.POS_REGEX_PATTERNS``):

            * noun phrase: r'<DET>? (<NOUN>+ <ADP|CONJ>)* <NOUN>+'
            * compound nouns: r'<NOUN>+'
            * verb phrase: r'<VERB>?<ADV>*<VERB>+'
            * prepositional phrase: r'<PREP> <DET>? (<NOUN>+<ADP>)* <NOUN>+'

    Yields:
        ``spacy.Span``: the next span of consecutive tokens from ``doc`` whose
        parts-of-speech match ``pattern``, in order of apperance
    """
    # standardize and transform the regular expression pattern...
    pattern = re.sub(r'\s', '', pattern)
    pattern = re.sub(r'<([A-Z]+)\|([A-Z]+)>', r'( (\1|\2))', pattern)
    pattern = re.sub(r'<([A-Z]+)>', r'( \1)', pattern)

    tags = ' ' + ' '.join(tgs)

    for m in re.finditer(pattern, tags):
        yield doc[tags[0:m.start()].count(' '):tags[0:m.end()].count(' ')]

def filter_noun_adj_from_phrase(sent):
    noun_phrases = []
    nonalpha = re.compile(r"[a-zA-Z0-9.@#_:)(-،‘]")
    sent = re.sub(nonalpha,'',sent).strip()#remove english alphabets
    tags = CRFTagger.pos_tag(sent)
    words,tgs = get_words_from_tags(tags)
    print(words)
    print(tgs)
    patern = r'<ADJ|A>*<NN|PN>+'
    lists = pos_regex_matches(words, patern,tgs)
    for j in lists:
        noun_phrases.append(" ".join(j))
  
    return noun_phrases

In [11]:
document_num = 50
doc_sample = dataset.iloc[document_num].Title

print("Original document: ")
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print("\n\nTokenized and lemmatized document: ")
print(preprocess_func(doc_sample))


Original document: 
['مظفرگڑھ,کشمیری', 'عوام', 'سے', 'اظہار', 'یکجہتی', 'کیلئی', 'ریلیاں', 'نکالی', 'گئیں']


Tokenized and lemmatized document: 
مظفرگڑھ کشمیری عوام اظہار یکجہتی کیلئی ریلیاں نکالی گئیں


In [12]:
dataset['preprocessed_text'] = dataset['Title'].apply(preprocess_func)

In [13]:
dataset['NounChunks'] = dataset['Title'].apply(filter_noun_adj_from_phrase)

['سی', 'پیک', 'منصوبہ', 'ختمآئی', 'ایم', 'ایف', 'کی', 'شرائط', 'پرعملدرآمد', 'شروع', 'امریکہ', 'کی', 'خوشنودی', 'کے', 'لیے', 'پاکستانی', 'مفاد', 'پسِ', 'پشت', 'ڈال', 'دیا', 'گیا']
['A', 'NN', 'NN', 'PN', 'PN', 'PN', 'P', 'NN', 'NN', 'NN', 'PN', 'P', 'NN', 'P', 'ADV', 'ADJ', 'NN', 'NN', 'NN', 'VB', 'AA', 'AA']
['حکومت', 'نے', 'سی', 'پیک', 'کے', 'تحت', 'اکانامک', 'زونز', 'پر', 'توجہ', 'مرکوز', 'رکھی', 'ہے', 'رشکئی', 'اکنامک', 'زون', 'گراونڈ', 'بریکینگ', 'کے', 'لیے', 'مکمل', 'ہے', 'آئندہ', 'ماہ', 'افتتاح', 'کردیا', 'جائے', 'گا', 'وفاقی', 'وزیر', 'برائے', 'منصوبہ', 'بندی', 'مزید']
['NN', 'P', 'A', 'NN', 'P', 'NN', 'PN', 'PN', 'P', 'NN', 'ADJ', 'VB', 'TA', 'PN', 'PN', 'PN', 'PN', 'PN', 'P', 'ADV', 'ADJ', 'VB', 'ADJ', 'NN', 'NN', 'VB', 'AA', 'TA', 'ADJ', 'NN', 'ADV', 'NN', 'NN', 'ADJ']
['سی', 'پیک', 'کاموجودہ', 'مرحلہ', 'پاکستانی', 'عوام', 'کیلئے', 'معاشی', 'فوائد', 'لے', 'کر', 'آئے', 'گا', 'سی', 'پیک', 'اب', 'دوسرے', 'مرحلہ', 'میں', 'داخل', 'ہو', 'گیا', 'ہے', 'جس', 'میں', 'غربت', 'کے', 'خا

In [14]:
dataset

,DateTime,Source,Title,lang,search_keywords,Label,preprocessed_text,NounChunks
0,2019-09-20 00:54:51,UrduPoint,سی پیک منصوبہ ختم،آئی ایم ایف کی شرائط پرعملدر...,ur,1,CPEC,پیک منصوبہ ایم ایف شرائط عمل درآمد شروع امریکہ...,"[سی پیک منصوبہ ختمآئی ایم ایف, شرائط پرعملدرآم..."
1,2019-09-20 05:45:20,UrduPoint,حکومت نے سی پیک کے تحت اکانامک زونز پر توجہ مر...,ur,1,CPEC,حکومت پیک اکانامک زونز توجہ مرکوز رشکئی اکنامک...,"[حکومت, سی پیک, تحت اکانامک زونز, توجہ, رشکئی ..."
2,2019-09-20 18:53:21,UrduPoint,سی پیک کاموجودہ مرحلہ پاکستانی عوام کیلئے معاش...,ur,1,CPEC,پیک کاموجودہ مرحلہ پاکستانی عوام کیلئے معاشی ف...,"[سی پیک, کاموجودہ مرحلہ, پاکستانی عوام, معاشی ..."
3,2019-09-21 09:28:12,UrduPoint,وزیر اعلیٰ خیبر پختونخوا محمود خان کا جمعہ کے ...,ur,1,CPEC,وزیر اعلی خیبر پختونخوا محمود خان جمعہ روز چتر...,"[وزیر, اعلیٰ خیبر پختونخوا محمود خان, جمعہ, رو..."
4,2019-09-23 06:40:21,UrduPoint,غربت کی شرح میں کمی سی پیک دوسرے مرحلے کی اہم...,ur,1,CPEC,غربت شرح کمی پیک مرحلے اہم ترجیحات شامل,"[غربت, شرح, کمی, سی پیک, دوسرے مرحلے, اہم ترجی..."
5,2019-09-23 06:40:21,UrduPoint,حکومت عوام کو ریلیف کی فراہمی کے لئے پر عزم ہے...,ur,1,CPEC,حکومت عوام ریلیف فراہمی عزم پیک اتھارٹی,"[حکومت عوام, ریلیف, فراہمی, لئے, عزم, سی پیک ا..."
6,2019-09-24 06:20:17,UrduPoint,چین پاکستان اقتصادی راہداری منصوبہ کے دوسرے مر...,ur,1,CPEC,چین پاکستان اقتصادی راہداری منصوبہ مرحلہ غربت ...,"[چین پاکستان, اقتصادی راہداری منصوبہ, دوسرے مر..."
7,2019-10-07 11:01:26,UrduPoint,وزیر اعظم عمران خان کے دورہ چین سے دونوں ممالک...,ur,1,CPEC,وزیر اعظم عمران خان دورہ چین ممالک مابین تعلقا...,"[وزیر اعظم عمران خان, دورہ چین, ممالک, مابین ت..."
8,2019-10-07 11:08:21,ExpressNews,صدارتی آرڈیننس کے ذریعے سی پیک اتھارٹی قائم کر...,ur,1,CPEC,صدارتی آرڈیننس پیک اتھارٹی قائم فیصلہ,"[صدارتی آرڈیننس, ذریعے, سی پیک اتھارٹی, فیصلہ]"
9,2019-10-07 15:09:59,UrduPoint,خیبرپختونخوا حکومت کی تمام متعلقہ محکموں کو تر...,ur,1,CPEC,خیبر پختونخوا حکومت متعلقہ محکموں ترقیاتی منصو...,"[خیبرپختونخوا حکومت, متعلقہ محکموں, ترقیاتی من..."


In [50]:
cnt_vectorizer = TfidfVectorizer(min_df=0.02)
vec = cnt_vectorizer.fit_transform(dataset['preprocessed_text'])


In [51]:
#elbow curve for optimal clusters
#Elbow curve method
X = vec.todense()
Sum_of_squared_distances = []
K = range(1,30)
for k in K:
    print('K is %s'%k)
    km = KMeans(n_clusters=k,random_state=3)
    km = km.fit(X)
    Sum_of_squared_distances.append(km.inertia_)


# In[25]:


kn = KneeLocator(K, Sum_of_squared_distances, curve='convex', direction='decreasing')
print(kn.knee)

K is 1
K is 2
K is 3
K is 4
K is 5
K is 6
K is 7
K is 8
K is 9
K is 10
K is 11
K is 12
K is 13
K is 14
K is 15
K is 16
K is 17
K is 18
K is 19
K is 20
K is 21
K is 22
K is 23
K is 24
K is 25
K is 26
K is 27
K is 28
K is 29
13


In [52]:
km = KMeans(kn.knee,random_state=3)
km.fit(X)
labels = km.labels_
dataset['km_label'] = labels

In [53]:
print(purity_score(dataset.Label,dataset['km_label']))

0.9807692307692307


In [54]:
order_centroids = km.cluster_centers_.argsort()[:, ::-1]
terms = cnt_vectorizer.get_feature_names()

In [55]:
for i in range(kn.knee):
    print("Cluster %d:" % i),
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind])

Cluster 0:
 کارروائی
 فوجی
 شام
 ترکی
 غاز
 لاروا
 عمل
 اجلاس
 افراد
 شہریوں
Cluster 1:
 ڈی
 ڈینگی
 ہسپتال
 لاہور
 مریضوں
 دورہ
 خانیوال
 بھرپور
 کار
 جائزہ
Cluster 2:
 ڈینگی
 حکومت
 اقدامات
 بچاؤ
 کیلئے
 سیمینار
 ملک
 مہم
 پنجاب
 لاروا
Cluster 3:
 پیک
 اتھارٹی
 حکام
 تعاون
 چین
 جاری
 عوام
 حکومت
 وزیراعظم
 پاکستان
Cluster 4:
 پیک
 چین
 وزیر
 پاکستان
 وفاقی
 حکومت
 منصوبوں
 وزیراعظم
 دورہ
 خان
Cluster 5:
 جاری
 ضلعی
 انتظامیہ
 ڈینگی
 صفائی
 مہم
 رپورٹ
 انسداد
 غاز
 ایم
Cluster 6:
 باد
 اسلام
 ڈاکٹر
 ڈینگی
 عمران
 افراد
 کشمیر
 انتظامیہ
 حوالے
 خان
Cluster 7:
 یکجہتی
 اظہار
 کشمیریوں
 ریلی
 کیلئے
 کشمیری
 کشمیر
 انعقاد
 عوام
 اہتمام
Cluster 8:
 سری
 لنکن
 ٹیم
 سیریز
 ٹی
 کرکٹ
 پاکستان
 میچ
 ٹونٹی
 پاک
Cluster 9:
 صحت
 ڈینگی
 وائرس
 محکمہ
 عوام
 وزیر
 ڈاکٹر
 ہسپتال
 سرکاری
 کیلئے
Cluster 10:
 تعداد
 مریضوں
 ڈینگی
 ہزار
 خیبر
 پنجاب
 راولپنڈی
 پختونخوا
 قابو
 مریض
Cluster 11:
 ڈپٹی
 کمشنر
 ڈینگی
 راولپنڈی
 انسداد
 ممکن
 کردار
 ضلع
 خاتمہ
 سیف
Cluster 12:
 سیکرٹری
 حال
 انسداد
 ڈینگی
 صدا

In [73]:
#get the closest documents to each cluster
from sklearn.neighbors import NearestNeighbors
neigh = NearestNeighbors(n_neighbors=5,metric='euclidean')
neigh.fit(X)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='euclidean',
         metric_params=None, n_jobs=1, n_neighbors=5, p=2, radius=1.0)

In [78]:
for i in range(0,kn.knee):
    nearest_docs_ids = neigh.kneighbors([km.cluster_centers_[i]])[1][0]
    docs_near_to_clusters =[]
    for x in nearest_docs_ids:
        print(x)
        docs_near_to_clusters.append(dataset.iloc[x]['Title'])
    for x in docs_near_to_clusters:
        print(x)
        print("========")

40
33
34
37
35
سلامتی کونسل کے رکن یورپی ممالک کا ترکی سے شام میں فوجی کارروائی روکنے کا مطالبہ
ترکی کی شام میں فوجی کارروائی کا آغاز
ترکی نے شام میں فوجی کارروائی کا آغاز کر دیا

ْ راس العین سے نقل مکانی کرنے والے افراد جنوبی علاقے الحسکہ کی طرف جنوب کی طرف جا رہے ہیں

یورپی یونین شام میں مجوزہ ’’سیف زون‘‘ کے قیام کے لیے کوئی رقم نہیں دے گی
178
251
175
215
156
آزادکشمیر میں 1085لوگ ڈینگی سے متاثر ہوئے، کوآرڈنیٹر نیشنل پروگرام وڈائریکٹر سی ڈی سی
ڈی سی لاہور اصغر جوئیہ کی این ڈی بوائز ہائی سکول اچھرہ میں ڈینگی سیمینار میں شرکت
ڈی سی لاہورکا سروسز ہسپتال کا دورہ، ڈینگی انتظامات کا جائزہ لیا
ڈی سی لاہور کی ہدایت پرافسران کے فیلڈ میں ڈینگی کے حوالے سے دورے
ڈی سی لاہور اصغر جوئیہ کاکوئینز روڈ کا دورہ ،انسدادِ ڈینگی میں ٹیموں کی کارکردگی کو چیک کیا
205
254
182
206
237
اربن یونین کونسلز میں آنے والے کھیتوں میں بھی ڈینگی سرویلنس کی جائے، عامرخٹک
احتیاط سے ڈینگی سے بچا جا سکتا ہے
ڈینگی ایک جان لیواء مرض ہے ڈینگی سے گھبرانے کی نہیں احیتاط کی ضرورت ہے
چکوال ،چمکیلی دھوپ نکلتے ہی ڈینگی کے وار

سلامتی کونسل کے رکن یورپی ممالک کا ترکی سے شام میں فوجی کارروائی روکنے کا مطالبہ
ترکی کی شام میں فوجی کارروائی کا آغاز
ترکی نے شام میں فوجی کارروائی کا آغاز کر دیا

ْ راس العین سے نقل مکانی کرنے والے افراد جنوبی علاقے الحسکہ کی طرف جنوب کی طرف جا رہے ہیں

یورپی یونین شام میں مجوزہ ’’سیف زون‘‘ کے قیام کے لیے کوئی رقم نہیں دے گی


In [110]:
vocab = list(terms)

In [111]:
len(vocab)

112

In [112]:
n_clust = kn.knee

In [113]:
n_clust

13

In [118]:
for i in range(0,n_clust):
    df_sub = dataset[dataset.km_label == i]
    df_sub = df_sub['Title_Mod']
    df_sub.to_csv('Cluster'+str(i)+'.txt',index=False,encoding='utf-8')

In [19]:
for i in range(0,kn.knee):
    df_sub = dataset[dataset.km_label == str(i)]
    print(df_sub.shape)
    txt = df_sub.Title
    df_sub

,DateTime,Source,Title,lang,search_keywords,Label,preprocessed_text,NounChunks,km_label
0,2019-09-20 00:54:51,UrduPoint,سی پیک منصوبہ ختم،آئی ایم ایف کی شرائط پرعملدر...,ur,1,CPEC,پیک منصوبہ ایم ایف شرائط عمل درآمد شروع امریکہ...,"[سی پیک منصوبہ ختمآئی ایم ایف, شرائط پرعملدرآم...",3
1,2019-09-20 05:45:20,UrduPoint,حکومت نے سی پیک کے تحت اکانامک زونز پر توجہ مر...,ur,1,CPEC,حکومت پیک اکانامک زونز توجہ مرکوز رشکئی اکنامک...,"[حکومت, سی پیک, تحت اکانامک زونز, توجہ, رشکئی ...",4
2,2019-09-20 18:53:21,UrduPoint,سی پیک کاموجودہ مرحلہ پاکستانی عوام کیلئے معاش...,ur,1,CPEC,پیک کاموجودہ مرحلہ پاکستانی عوام کیلئے معاشی ف...,"[سی پیک, کاموجودہ مرحلہ, پاکستانی عوام, معاشی ...",3
3,2019-09-21 09:28:12,UrduPoint,وزیر اعلیٰ خیبر پختونخوا محمود خان کا جمعہ کے ...,ur,1,CPEC,وزیر اعلی خیبر پختونخوا محمود خان جمعہ روز چتر...,"[وزیر, اعلیٰ خیبر پختونخوا محمود خان, جمعہ, رو...",4
4,2019-09-23 06:40:21,UrduPoint,غربت کی شرح میں کمی سی پیک دوسرے مرحلے کی اہم...,ur,1,CPEC,غربت شرح کمی پیک مرحلے اہم ترجیحات شامل,"[غربت, شرح, کمی, سی پیک, دوسرے مرحلے, اہم ترجی...",3
5,2019-09-23 06:40:21,UrduPoint,حکومت عوام کو ریلیف کی فراہمی کے لئے پر عزم ہے...,ur,1,CPEC,حکومت عوام ریلیف فراہمی عزم پیک اتھارٹی,"[حکومت عوام, ریلیف, فراہمی, لئے, عزم, سی پیک ا...",3
6,2019-09-24 06:20:17,UrduPoint,چین پاکستان اقتصادی راہداری منصوبہ کے دوسرے مر...,ur,1,CPEC,چین پاکستان اقتصادی راہداری منصوبہ مرحلہ غربت ...,"[چین پاکستان, اقتصادی راہداری منصوبہ, دوسرے مر...",3
7,2019-10-07 11:01:26,UrduPoint,وزیر اعظم عمران خان کے دورہ چین سے دونوں ممالک...,ur,1,CPEC,وزیر اعظم عمران خان دورہ چین ممالک مابین تعلقا...,"[وزیر اعظم عمران خان, دورہ چین, ممالک, مابین ت...",4
8,2019-10-07 11:08:21,ExpressNews,صدارتی آرڈیننس کے ذریعے سی پیک اتھارٹی قائم کر...,ur,1,CPEC,صدارتی آرڈیننس پیک اتھارٹی قائم فیصلہ,"[صدارتی آرڈیننس, ذریعے, سی پیک اتھارٹی, فیصلہ]",3
9,2019-10-07 15:09:59,UrduPoint,خیبرپختونخوا حکومت کی تمام متعلقہ محکموں کو تر...,ur,1,CPEC,خیبر پختونخوا حکومت متعلقہ محکموں ترقیاتی منصو...,"[خیبرپختونخوا حکومت, متعلقہ محکموں, ترقیاتی من...",4


In [18]:
prob_clusters = []
for i in range(0,n_clust):
    prob_clusters.append(get_pd_of_cluster(dataset,i))

In [19]:
theta=[]
for i in range(0,n_clust):
    p_w = []
    for w in vocab:
        if w in prob_clusters[i].keys():
            p_w.append(prob_clusters[i][w])
        else:
            p_w.append(0)
    theta.append(p_w)

In [20]:
x = pd.DataFrame(theta)
x.columns = vocab

In [21]:
x.shape

(13, 112)

In [22]:
theta=x.values

In [23]:
theta.shape


(13, 112)

In [24]:
freq = vec.sum(axis=0)


In [25]:
freq = (freq/np.sum(freq))

In [26]:
freq.shape

(1, 112)

In [27]:
all_noun_chunks =[]


In [28]:
for x in dataset.NounChunks:
    for j in x:
        all_noun_chunks.append(j)
        
   
    

In [29]:
all_noun_chunks

['سی پیک منصوبہ ختمآئی ایم ایف',
 'شرائط پرعملدرآمد شروع امریکہ',
 'خوشنودی',
 'پاکستانی مفاد پسِ پشت',
 'حکومت',
 'سی پیک',
 'تحت اکانامک زونز',
 'توجہ',
 'رشکئی اکنامک زون گراونڈ بریکینگ',
 'آئندہ ماہ افتتاح',
 'وفاقی وزیر',
 'منصوبہ بندی',
 'سی پیک',
 'کاموجودہ مرحلہ',
 'پاکستانی عوام',
 'معاشی فوائد',
 'سی پیک',
 'دوسرے مرحلہ',
 'داخل',
 'غربت',
 'خاتمہ زراعت',
 'صنعتی تعاون',
 'توجہ',
 'وزیر',
 'اعلیٰ خیبر پختونخوا محمود خان',
 'جمعہ',
 'روز چترال',
 'سرکاری دورہ وزیر اعلیٰ',
 'سی پیک',
 'متبادل روٹ چترل',
 'متاثرہ ریشن بجلی گھر',
 'غربت',
 'شرح',
 'کمی',
 'سی پیک',
 'دوسرے مرحلے',
 'اہم ترجیحات',
 'حکومت عوام',
 'ریلیف',
 'فراہمی',
 'لئے',
 'عزم',
 'سی پیک اتھارٹی',
 'چین پاکستان',
 'اقتصادی راہداری منصوبہ',
 'دوسرے مرحلہ',
 'غربت',
 'خاتمے زراعت',
 'صنعتی تعاون',
 'خصوصی توجہ',
 'حکام',
 'سی پیک اتھارٹی',
 'وزیر اعظم عمران خان',
 'دورہ چین',
 'ممالک',
 'مابین تعلقات',
 'نئے دور',
 'آغاز',
 'سی پیک',
 'کام',
 'رفتار',
 'شاہد رشید',
 'صدارتی آرڈیننس',
 'ذریعے',
 'سی پیک اتھارٹی',

In [30]:
allnounchunks = pd.DataFrame({'NounChunks':all_noun_chunks})

In [31]:
allnounchunks.to_csv('allnounchunksUrduDS.csv',index=False)

In [32]:
#allnounchunks = pd.read_csv('allnounchunks20NewsGroup.csv')

In [33]:
all_noun_chunks = allnounchunks.NounChunks
print(len(all_noun_chunks))

1758


In [34]:
from collections import Counter

In [35]:
freq_all_noun_chunks = Counter(all_noun_chunks)

In [36]:
freq_all_noun_chunks.most_common(1000)

[('ڈینگی', 74),
 ('سی پیک', 22),
 ('پاکستان', 17),
 ('اظہار یکجہتی', 16),
 ('انعقاد', 15),
 ('حوالے', 14),
 ('کشمیریوں', 13),
 ('ریلی', 13),
 ('تعداد', 12),
 ('لئے', 11),
 ('سی پیک اتھارٹی', 11),
 ('اظہار', 11),
 ('سری لنکن ٹیم', 10),
 ('ڈینگی وائرس', 10),
 ('مریضوں', 9),
 ('ضلعی انتظامیہ', 8),
 ('پنجاب', 8),
 ('چین', 7),
 ('خاتمہ', 7),
 ('قابو', 7),
 ('حکام', 6),
 ('ڈپٹی کمشنر', 6),
 ('ڈی', 6),
 ('صفائی', 6),
 ('دورہ', 5),
 ('ملک', 5),
 ('وفاقی حکومت', 5),
 ('ترکی', 5),
 ('اسلام آباد', 5),
 ('وجہ', 5),
 ('تجاوز', 5),
 ('بچاؤ', 5),
 ('انسداد ڈینگی', 5),
 ('عوام', 5),
 ('انسداد ڈینگی مہم', 5),
 ('دورہ چین', 4),
 ('آغاز', 4),
 ('فیصلہ', 4),
 ('سی آر', 4),
 ('درمیان', 4),
 ('شام', 4),
 ('فوجی کارروائی', 4),
 ('قرارداد', 4),
 ('پنجاب اسمبلی', 4),
 ('ساتھ اظہار یکجہتی', 4),
 ('جانب', 4),
 ('ساتھ', 4),
 ('عیادت', 4),
 ('اقدامات', 4),
 ('آگاہی', 4),
 ('تصدیق', 4),
 ('سرکاری ہسپتالوں', 4),
 ('کنٹرول', 4),
 ('کارکردگی', 4),
 ('جائزہ', 4),
 ('بڑے پیمانے', 4),
 ('داخل', 3),
 ('غربت', 3),
 ('ص

In [37]:
# Take top 1000 frequent noun phrases as Candidate labels as suggested by Mei et al.
top_1000_candid = freq_all_noun_chunks.most_common(1000)

In [38]:
candidate_labels_with_freq = []
for u,v in top_1000_candid:
    candidate_labels_with_freq.append((u,v))


In [39]:
candidate_labels_with_freq

[('ڈینگی', 74),
 ('سی پیک', 22),
 ('پاکستان', 17),
 ('اظہار یکجہتی', 16),
 ('انعقاد', 15),
 ('حوالے', 14),
 ('کشمیریوں', 13),
 ('ریلی', 13),
 ('تعداد', 12),
 ('لئے', 11),
 ('سی پیک اتھارٹی', 11),
 ('اظہار', 11),
 ('سری لنکن ٹیم', 10),
 ('ڈینگی وائرس', 10),
 ('مریضوں', 9),
 ('ضلعی انتظامیہ', 8),
 ('پنجاب', 8),
 ('چین', 7),
 ('خاتمہ', 7),
 ('قابو', 7),
 ('حکام', 6),
 ('ڈپٹی کمشنر', 6),
 ('ڈی', 6),
 ('صفائی', 6),
 ('دورہ', 5),
 ('ملک', 5),
 ('وفاقی حکومت', 5),
 ('ترکی', 5),
 ('اسلام آباد', 5),
 ('وجہ', 5),
 ('تجاوز', 5),
 ('بچاؤ', 5),
 ('انسداد ڈینگی', 5),
 ('عوام', 5),
 ('انسداد ڈینگی مہم', 5),
 ('دورہ چین', 4),
 ('آغاز', 4),
 ('فیصلہ', 4),
 ('سی آر', 4),
 ('درمیان', 4),
 ('شام', 4),
 ('فوجی کارروائی', 4),
 ('قرارداد', 4),
 ('پنجاب اسمبلی', 4),
 ('ساتھ اظہار یکجہتی', 4),
 ('جانب', 4),
 ('ساتھ', 4),
 ('عیادت', 4),
 ('اقدامات', 4),
 ('آگاہی', 4),
 ('تصدیق', 4),
 ('سرکاری ہسپتالوں', 4),
 ('کنٹرول', 4),
 ('کارکردگی', 4),
 ('جائزہ', 4),
 ('بڑے پیمانے', 4),
 ('داخل', 3),
 ('غربت', 3),
 ('ص

In [40]:
def string_contains(str1,str2):
    lst1 = str1.split(' ')
    lst2 = str2.split(' ')

    if len(lst1) <= len(lst2):
        return lst1 == lst2[:len(lst1)]

    return False
def compute_c_value(row):
    #if non-nested
    if len(row.Containing_Strings) == 0:
        CValue = math.log2(len(row.Candidate_Label.split(' '))) * row.Frequency
    else:
        sum_f=0
        for i in range(0,len(row.Containing_Strings)):
             sum_f = sum_f+row.Containing_Strings[i][1]
        CValue = math.log2(len(row.Candidate_Label.split(' '))) * (row.Frequency - 1/len(row.Containing_Strings) * sum_f)
    return CValue
def find_if_short_term(row,df):
#To introduce the notion of “termhood” [11], we define that a
#term t is a short term if it is nested in a longer term t' that has a bigger value
#for some base measure of termhood (e.g., c-value).
    short_term = False
    if len(row.Containing_Strings) > 0:
        for i in range(0,len(row.Containing_Strings)):
            x = df[df['Candidate_Label']==row.Containing_Strings[i][0]].index.values
            index_of_bigger_term=x[0]
            if df.iloc[index_of_bigger_term]['CValue'] > row.CValue:
                short_term = True
                break
    return short_term

In [41]:
#Computing C-values
containing_strings = []
candidate_labels=[]
freq_candidate_labels=[]
for i in range(0,len(candidate_labels_with_freq)):
    candidate_labels.append(candidate_labels_with_freq[i][0])
    freq_candidate_labels.append(candidate_labels_with_freq[i][1])
    containing_strings_for_i=[]
    for j in range(0,len(candidate_labels_with_freq)):
        if string_contains(candidate_labels_with_freq[i][0],candidate_labels_with_freq[j][0]) and len(candidate_labels_with_freq[j][0]) > len(candidate_labels_with_freq[i][0]):
            containing_strings_for_i.append(candidate_labels_with_freq[j])
    containing_strings.append(containing_strings_for_i)

In [42]:
c_value_pd = pd.DataFrame({'Candidate_Label':candidate_labels,'Frequency':freq_candidate_labels,'Containing_Strings':containing_strings})

In [43]:
c_value_pd

,Candidate_Label,Frequency,Containing_Strings
0,ڈینگی,74,"[(ڈینگی وائرس, 10), (ڈینگی مریضوں, 3), (ڈینگی ..."
1,سی پیک,22,"[(سی پیک اتھارٹی, 11), (سی پیک اتھارٹی آرڈینن..."
2,پاکستان,17,"[(پاکستان غربت, 1), (پاکستان کےخلاف ٹی ٹونٹی س..."
3,اظہار یکجہتی,16,[]
4,انعقاد,15,[]
5,حوالے,14,[]
6,کشمیریوں,13,[]
7,ریلی,13,[]
8,تعداد,12,[]
9,لئے,11,"[(لئے اقوام متحدہ, 1), (لئے گورنمنٹ گرلزکالج و..."


In [44]:
c_values =[]
for index,row in c_value_pd.iterrows():
    c_values.append(compute_c_value(row))


In [45]:
c_value_pd['CValue'] = c_values

In [46]:
is_short_term =[]
for index,row in c_value_pd.iterrows():
    is_short_term.append(find_if_short_term(row,c_value_pd))


In [47]:
c_value_pd['ShortTerm'] = is_short_term

In [48]:
c_value_pd

,Candidate_Label,Frequency,Containing_Strings,CValue,ShortTerm
0,ڈینگی,74,"[(ڈینگی وائرس, 10), (ڈینگی مریضوں, 3), (ڈینگی ...",0.000000,True
1,سی پیک,22,"[(سی پیک اتھارٹی, 11), (سی پیک اتھارٹی آرڈینن...",18.800000,False
2,پاکستان,17,"[(پاکستان غربت, 1), (پاکستان کےخلاف ٹی ٹونٹی س...",0.000000,True
3,اظہار یکجہتی,16,[],16.000000,False
4,انعقاد,15,[],0.000000,False
5,حوالے,14,[],0.000000,False
6,کشمیریوں,13,[],0.000000,False
7,ریلی,13,[],0.000000,False
8,تعداد,12,[],0.000000,False
9,لئے,11,"[(لئے اقوام متحدہ, 1), (لئے گورنمنٹ گرلزکالج و...",0.000000,True


In [49]:
def t_order(freq,theta,c_value_pd,k,totaltopics):
    # bob la liste qui contient tous les numéros de topic (lignes), sauf celui du topic selectionné
    bob =list(range(totaltopics))
    del bob[k]
    
    # M la moyenne des proba d'avoir le mot, sachant qu'on est pas dans le topic k
    M=theta[bob,:].mean(axis=0)
    #print(M)
    #W matrice qui contient le score de chaque mot pour chaque topic
    W=np.log(theta[k]/M)
    
    score=[]
    for indice in range (len(c_value_pd)):
        if c_value_pd.iloc[indice]['ShortTerm'] == True:
            score.append(0)
        else:
            lCandidat = c_value_pd.Candidate_Label

            candidat=lCandidat[indice].split(" ")
            sum_w=[]
            #print(candidat)
            for w in candidat:
                if w in vocab:
                    i=vocab.index(w)
                    sum_w.append(W[i])
                else:
                    sum_w.append(0)
            #i=id2word.doc2idx(candidat)
            # supprime les -1 (qui signifie pas trouvé)
            #i[:] = [v for v in i if v != -1]
            score.append(np.sum(sum_w))

    #topValue, topCandidate = top10Score(score,lCandidat)
    dicti=pd.DataFrame({'Labels':lCandidat,'Score':score})
  
    return dicti

def m_order(freq,theta,lCandidat,k,totaltopics):
    """
    Calculate the M-Order Relevance

    Parameters:
    ----------
    freq : Array containing the frequency of occurrences of each word in the whole corpus
    theta : Array containing the frequency of occurrences of each word in each topic
    lcandidat: Array containing each label candidate
    k : The number of the topic
    
    Returns:
    -------
    topCandidate : Array containing the name of the top 10 score candidate for a given topic
    """
    # bob la liste qui contient tous les numéros de topic (lignes), sauf celui du topic selectionné
    bob =list(range(totaltopics))
    del bob[k]
    
    # M la moyenne des proba d'avoir le mot, sachant qu'on est pas dans le topic k
    M=theta[bob,:].mean(axis=0)
    #print(M)
    #W matrice qui contient le score de chaque mot pour chaque topic
    W=np.log(theta[k]/M)
    
    score=[]
    for indice in range (len(lCandidat)):
        candidat=lCandidat[indice].split(" ")
        sum_w=[]
        #print(candidat)
        for w in candidat:
            if w in vocab:
                i=vocab.index(w)
                sum_w.append(W[i])
            else:
                sum_w.append(0)
        #i=id2word.doc2idx(candidat)
        # supprime les -1 (qui signifie pas trouvé)
        #i[:] = [v for v in i if v != -1]
        score.append(np.sum(sum_w))
        
    #topValue, topCandidate = top10Score(score,lCandidat)
    dicti=pd.DataFrame({'Labels':lCandidat,'Score':score})
  
    return dicti

def zero_order(freq,theta,lCandidat,NumTopic):
    """
    Calculate the Zero-Order Relevance

    Parameters:
    ----------
    freq : Array containing the frequency of occurrences of each word in the whole corpus
    theta : Array containing the frequency of occurrences of each word in each topic
    lcandidat: Array containing each label candidate
    NumTopic : The number of the topic
    
    Returns:
    -------
    topCandidate : Array containing the name of the top 10 score candidate for a given topic
    """
    
    #W matrice qui contient le score de chaque mot pour chaque topic
    W=np.log(theta/freq)
    
    # score des tous les candidats pour le topic NumTopic
    score=[]
    
    for indice in range (len(lCandidat)):
        candidat=lCandidat[indice].split(" ")
        sum_w=[]
        #print(candidat)
        for w in candidat:
            if w in vocab:
                i=vocab.index(w)
                sum_w.append(W[k,i])
            else:
                sum_w.append(0)
        #i=id2word.doc2idx(candidat)
        # supprime les -1 (qui signifie pas trouvé)
        #i[:] = [v for v in i if v != -1]
        
        score.append(np.sum(sum_w))
        
    #topValue, topCandidate = top10Score(score,lCandidat)
    dicti=pd.DataFrame({'Labels':lCandidat,'Score':score})
  
    return dicti



In [51]:
labels_zero_order=[]
for k in range(0,kn.knee):
    print("Topic %d"%k)
    candid_for_topic_k = zero_order(freq,theta,c_value_pd.Candidate_Label,k)
    candid_for_topic_k = candid_for_topic_k.sort_values('Score',ascending=False)
    print(candid_for_topic_k.head(3))
    print("=======================")
    labels_zero_order.append(candid_for_topic_k.iloc[0]['Labels'])

Topic 0


C:\Users\Zarmeen\Anaconda2\envs\py36\lib\site-packages\ipykernel_launcher.py:101: RuntimeWarning: divide by zero encountered in log


                Labels     Score
372           ترکی شام  5.027776
67   ترک فوجی کارروائی  4.933145
41       فوجی کارروائی  4.933145
Topic 1
                                    Labels     Score
759  ڈی ایچ کیو ہسپتال وہاڑی کادورہ ہسپتال  4.439810
814                            ڈی جی نرسنگ  1.756092
22                                      ڈی  1.756092
Topic 2
                    Labels     Score
84              ڈینگی مچھر  0.982508
201  ممکنہ مضراثرات او راس  0.936505
232          ڈینگی سیمینار  0.870673
Topic 3
                      Labels     Score
344      سی پیک اتھارٹی بارے  3.473964
118  سی پیک اتھارٹی آرڈیننس  3.473964
10            سی پیک اتھارٹی  3.473964
Topic 4
                  Labels     Score
103   وزیراعظم عمران خان  2.955903
100  وزیر اعظم عمران خان  2.630050
274       سی پیک منصوبوں  1.915823
Topic 5
                   Labels     Score
981   ضلعی انتظامیہ لاہور  3.064963
15          ضلعی انتظامیہ  2.988207
669  ضلعی انتظامیہ لاہو ر  2.988207
Topic 6
               Label

In [52]:
labels_zero_order

['ترکی شام',
 'ڈی ایچ کیو ہسپتال وہاڑی کادورہ ہسپتال',
 'ڈینگی مچھر',
 'سی پیک اتھارٹی بارے',
 'وزیراعظم عمران خان',
 'ضلعی انتظامیہ لاہور',
 'پریشان عمران خان',
 'کشمیری عوام کیساتھ اظہار یکجہتی',
 'پاک سری لنکن ٹی ٹونٹی کرکٹ سیریز سکیورٹی',
 'محکمہ صحت ڈینگی وائرس',
 'خیبر پختونخوا',
 'ڈپٹی کمشنر محمد سیف انور جپہ',
 'زیر صدارت انسداد ڈینگی']

In [53]:
labels_m_order=[]
for k in range(0,kn.knee):
    print("Topic %d"%k)
    candid_for_topic_k = m_order(freq,theta,c_value_pd.Candidate_Label,k,kn.knee)
    candid_for_topic_k = candid_for_topic_k.sort_values('Score',ascending=False)
    print(candid_for_topic_k.head(3))
    print("=======================")
    labels_m_order.append(candid_for_topic_k.iloc[0]['Labels'])

Topic 0


C:\Users\Zarmeen\Anaconda2\envs\py36\lib\site-packages\ipykernel_launcher.py:61: RuntimeWarning: divide by zero encountered in true_divide
C:\Users\Zarmeen\Anaconda2\envs\py36\lib\site-packages\ipykernel_launcher.py:61: RuntimeWarning: divide by zero encountered in log


                               Labels  Score
67                  ترک فوجی کارروائی    inf
373  کرد جنگجوؤں کیخلاف فوجی کارروائی    inf
372                          ترکی شام    inf
Topic 1
                                    Labels      Score
759  ڈی ایچ کیو ہسپتال وہاڑی کادورہ ہسپتال  12.992933
893                  ڈی جی صحت آزاد کشمیر   6.261112
814                            ڈی جی نرسنگ   6.167146
Topic 2
                               Labels  Score
772            سی ای او ہیلتھ خانیوال    inf
201             ممکنہ مضراثرات او راس    inf
992  ہیںسی ای او ہیلتھ ڈاکٹر ارشد ملک    inf
Topic 3
                         Labels     Score
20                         حکام       inf
322  وزیراعظم عمران خان اتھارٹی  8.813702
118     سی پیک اتھارٹی آرڈیننس  8.396886
Topic 4
                         Labels     Score
103          وزیراعظم عمران خان  6.605659
100         وزیر اعظم عمران خان  6.191853
859  پی ٹی آئی حکومت عمران خان  5.305260
Topic 5
                     Labels     Score
981     ضل

In [54]:
labels_t_order=[]
for k in range(0,kn.knee):
    print("Topic %d"%k)
    candid_for_topic_k = t_order(freq,theta,c_value_pd,k,kn.knee)
    candid_for_topic_k = candid_for_topic_k.sort_values('Score',ascending=False)
    print(candid_for_topic_k.head(3))
    print("=======================")
    labels_t_order.append(candid_for_topic_k.iloc[0]['Labels'])

Topic 0


C:\Users\Zarmeen\Anaconda2\envs\py36\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning: divide by zero encountered in true_divide
  # Remove the CWD from sys.path while we load stuff.
C:\Users\Zarmeen\Anaconda2\envs\py36\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning: divide by zero encountered in log
  # Remove the CWD from sys.path while we load stuff.


                Labels  Score
372           ترکی شام    inf
67   ترک فوجی کارروائی    inf
41       فوجی کارروائی    inf
Topic 1
                                    Labels      Score
759  ڈی ایچ کیو ہسپتال وہاڑی کادورہ ہسپتال  12.992933
893                  ڈی جی صحت آزاد کشمیر   6.261112
814                            ڈی جی نرسنگ   6.167146
Topic 2
                     Labels  Score
772  سی ای او ہیلتھ خانیوال    inf
641          سی او راولپنڈی    inf
201   ممکنہ مضراثرات او راس    inf
Topic 3
                         Labels     Score
20                         حکام       inf
322  وزیراعظم عمران خان اتھارٹی  8.813702
344         سی پیک اتھارٹی بارے  8.396886
Topic 4
                         Labels     Score
100         وزیر اعظم عمران خان  6.191853
859  پی ٹی آئی حکومت عمران خان  5.305260
93                   وفاقی وزیر  5.234032
Topic 5
                     Labels     Score
981     ضلعی انتظامیہ لاہور  6.758603
210  ضلعی انتظامیہ راولپنڈی  5.872559
669    ضلعی انتظامیہ لاہو ر  5.670

In [253]:
dataset.to_csv('dataset_urdu_kmeans.csv',sep='\t',index=False)

In [254]:
dataset

,DateTime,Source,Title,lang,search_keywords,Label,preprocessed_text,NounChunks,km_label
0,2019-09-20 00:54:51,UrduPoint,سی پیک منصوبہ ختم،آئی ایم ایف کی شرائط پرعملدر...,ur,1,CPEC,پیک منصوبہ ایم ایف شرائط عمل درآمد شروع امریکہ...,"[سی پیک منصوبہ ختمآئی ایم ایف, شرائط پرعملدرآم...",3
1,2019-09-20 05:45:20,UrduPoint,حکومت نے سی پیک کے تحت اکانامک زونز پر توجہ مر...,ur,1,CPEC,حکومت پیک اکانامک زونز توجہ مرکوز رشکئی اکنامک...,"[حکومت, سی پیک, تحت اکانامک زونز, توجہ, رشکئی ...",4
2,2019-09-20 18:53:21,UrduPoint,سی پیک کاموجودہ مرحلہ پاکستانی عوام کیلئے معاش...,ur,1,CPEC,پیک کاموجودہ مرحلہ پاکستانی عوام کیلئے معاشی ف...,"[سی پیک, کاموجودہ مرحلہ, پاکستانی عوام, معاشی ...",3
3,2019-09-21 09:28:12,UrduPoint,وزیر اعلیٰ خیبر پختونخوا محمود خان کا جمعہ کے ...,ur,1,CPEC,وزیر اعلی خیبر پختونخوا محمود خان جمعہ روز چتر...,"[وزیر, اعلیٰ خیبر پختونخوا محمود خان, جمعہ, رو...",4
4,2019-09-23 06:40:21,UrduPoint,غربت کی شرح میں کمی سی پیک دوسرے مرحلے کی اہم...,ur,1,CPEC,غربت شرح کمی پیک مرحلے اہم ترجیحات شامل,"[غربت, شرح, کمی, سی پیک, دوسرے مرحلے, اہم ترجی...",3
5,2019-09-23 06:40:21,UrduPoint,حکومت عوام کو ریلیف کی فراہمی کے لئے پر عزم ہے...,ur,1,CPEC,حکومت عوام ریلیف فراہمی عزم پیک اتھارٹی,"[حکومت عوام, ریلیف, فراہمی, لئے, عزم, سی پیک ا...",3
6,2019-09-24 06:20:17,UrduPoint,چین پاکستان اقتصادی راہداری منصوبہ کے دوسرے مر...,ur,1,CPEC,چین پاکستان اقتصادی راہداری منصوبہ مرحلہ غربت ...,"[چین پاکستان, اقتصادی راہداری منصوبہ, دوسرے مر...",3
7,2019-10-07 11:01:26,UrduPoint,وزیر اعظم عمران خان کے دورہ چین سے دونوں ممالک...,ur,1,CPEC,وزیر اعظم عمران خان دورہ چین ممالک مابین تعلقا...,"[وزیر اعظم عمران خان, دورہ چین, ممالک, مابین ت...",4
8,2019-10-07 11:08:21,ExpressNews,صدارتی آرڈیننس کے ذریعے سی پیک اتھارٹی قائم کر...,ur,1,CPEC,صدارتی آرڈیننس پیک اتھارٹی قائم فیصلہ,"[صدارتی آرڈیننس, ذریعے, سی پیک اتھارٹی, فیصلہ]",3
9,2019-10-07 15:09:59,UrduPoint,خیبرپختونخوا حکومت کی تمام متعلقہ محکموں کو تر...,ur,1,CPEC,خیبر پختونخوا حکومت متعلقہ محکموں ترقیاتی منصو...,"[خیبرپختونخوا حکومت, متعلقہ محکموں, ترقیاتی من...",4


## Proposed Topic labeling


In [34]:

def list_phrases_cluster(cluster_idx,df,key):
    df_sub = df[df[key] == cluster_idx]
    hashs = []
    for index,row in df_sub.iterrows():
        row.Noun_Phrases = row.NounChunks
        for x in row.Noun_Phrases:
            hashs.append(x)
    
    return collections.Counter(hashs)
def return_clusters_title(cluster_idx,dataset,col):
    hash_cluster1 = list_phrases_cluster(cluster_idx,dataset,col) 
    top10  = hash_cluster1.most_common(3)
    dataset_sub = dataset[dataset[col] == cluster_idx]
    cnt_vectorizer = TfidfVectorizer(ngram_range=(3,8))
    vec = cnt_vectorizer.fit_transform(dataset_sub['Title'])
    scores = (vec.toarray()) 
    # Getting top ranking features 
    sums = vec.sum(axis = 0) 
    data1 = [] 
    features = cnt_vectorizer.get_feature_names()
    for col, term in enumerate(features): 
        data1.append((term, sums[0, col] )) 
    ranking = pd.DataFrame(data1, columns = ['term', 'rank'])
    ranking['wCount'] = ranking['term'].apply(lambda x: len(x.split()))
    ranking = ranking.sort_values(['rank','wCount'],ascending=[False,True])
    print(ranking.head(5))
    my_top_4=[]
    for u,v in top10:
        my_top_4.append(u)
    my_top_4 = " ".join(my_top_4)
    my_top_4 = my_top_4.replace("_"," ")

    #print(my_top_4)
    test_list = (my_top_4.split())
    df = pd.DataFrame(columns=ranking.columns) #empty dataframe to add filtered rows
    for index,row in ranking.iterrows():
        # using list comprehension 
        # checking if string contains list element 
        res = all(ele in row['term'] for ele in test_list) 

        # print result 
        if res == True:
            df.loc[len(df)]=[row['term'],row['rank'],row['wCount']] 
    df = df.sort_values(['rank','wCount'],ascending=[False,True])
    #df = df.sort_values(['wCount'])
    if len(df)>0:
        return df.iloc[0]['term']
    else:
        return "No title constructed"

    
def return_clusters_title_sgram(cluster_idx,dataset,col):
    hash_cluster1 = list_phrases_cluster(cluster_idx,dataset,col) 
    top10  = hash_cluster1.most_common(3)
    dataset_sub = dataset[dataset[col] == cluster_idx]
    bigrams=[]
    for index,row in dataset_sub.iterrows():
        print(row.Title)
        SNgram_obj=SNgramExtractor(row['Title'],meta_tag='original',trigram_flag='yes',nlp_model=nlp)
        output=SNgram_obj.get_SNgram()
        print(output['SNBigram'])
        for x in output['SNBigram'].split():
            
            bigrams.append(x)
            
    my_top_4=[]
    for u,v in top10:
        my_top_4.append(u)
    my_top_4 = " ".join(my_top_4)
    my_top_4 = my_top_4.replace("_"," ")

    #print(my_top_4)
    test_list = (my_top_4.split())
    candid_labels=[]
    for x in bigrams:
        # using list comprehension 
        # checking if string contains list element 
        res = any(ele in x for ele in test_list) 

        # print result 
        if res == True:
            candid_labels.append(x) 
    return candid_labels()

In [18]:
dataset

,DateTime,Source,Title,lang,search_keywords,Label,preprocessed_text,NounChunks,km_label
0,2019-09-20 00:54:51,UrduPoint,سی پیک منصوبہ ختم،آئی ایم ایف کی شرائط پرعملدر...,ur,1,CPEC,پیک منصوبہ ایم ایف شرائط عمل درآمد شروع امریکہ...,"[سی پیک منصوبہ ختمآئی ایم ایف, شرائط پرعملدرآم...",3
1,2019-09-20 05:45:20,UrduPoint,حکومت نے سی پیک کے تحت اکانامک زونز پر توجہ مر...,ur,1,CPEC,حکومت پیک اکانامک زونز توجہ مرکوز رشکئی اکنامک...,"[حکومت, سی پیک, تحت اکانامک زونز, توجہ, رشکئی ...",4
2,2019-09-20 18:53:21,UrduPoint,سی پیک کاموجودہ مرحلہ پاکستانی عوام کیلئے معاش...,ur,1,CPEC,پیک کاموجودہ مرحلہ پاکستانی عوام کیلئے معاشی ف...,"[سی پیک, کاموجودہ مرحلہ, پاکستانی عوام, معاشی ...",3
3,2019-09-21 09:28:12,UrduPoint,وزیر اعلیٰ خیبر پختونخوا محمود خان کا جمعہ کے ...,ur,1,CPEC,وزیر اعلی خیبر پختونخوا محمود خان جمعہ روز چتر...,"[وزیر, اعلیٰ خیبر پختونخوا محمود خان, جمعہ, رو...",4
4,2019-09-23 06:40:21,UrduPoint,غربت کی شرح میں کمی سی پیک دوسرے مرحلے کی اہم...,ur,1,CPEC,غربت شرح کمی پیک مرحلے اہم ترجیحات شامل,"[غربت, شرح, کمی, سی پیک, دوسرے مرحلے, اہم ترجی...",3
5,2019-09-23 06:40:21,UrduPoint,حکومت عوام کو ریلیف کی فراہمی کے لئے پر عزم ہے...,ur,1,CPEC,حکومت عوام ریلیف فراہمی عزم پیک اتھارٹی,"[حکومت عوام, ریلیف, فراہمی, لئے, عزم, سی پیک ا...",3
6,2019-09-24 06:20:17,UrduPoint,چین پاکستان اقتصادی راہداری منصوبہ کے دوسرے مر...,ur,1,CPEC,چین پاکستان اقتصادی راہداری منصوبہ مرحلہ غربت ...,"[چین پاکستان, اقتصادی راہداری منصوبہ, دوسرے مر...",3
7,2019-10-07 11:01:26,UrduPoint,وزیر اعظم عمران خان کے دورہ چین سے دونوں ممالک...,ur,1,CPEC,وزیر اعظم عمران خان دورہ چین ممالک مابین تعلقا...,"[وزیر اعظم عمران خان, دورہ چین, ممالک, مابین ت...",4
8,2019-10-07 11:08:21,ExpressNews,صدارتی آرڈیننس کے ذریعے سی پیک اتھارٹی قائم کر...,ur,1,CPEC,صدارتی آرڈیننس پیک اتھارٹی قائم فیصلہ,"[صدارتی آرڈیننس, ذریعے, سی پیک اتھارٹی, فیصلہ]",3
9,2019-10-07 15:09:59,UrduPoint,خیبرپختونخوا حکومت کی تمام متعلقہ محکموں کو تر...,ur,1,CPEC,خیبر پختونخوا حکومت متعلقہ محکموں ترقیاتی منصو...,"[خیبرپختونخوا حکومت, متعلقہ محکموں, ترقیاتی من...",4


In [19]:
top_elements_topics = []
for i in range(0,kn.knee):
    hash_cluster1 = list_phrases_cluster(i,dataset,'km_label')
    top_elements_topics.append(hash_cluster1.most_common(10))

In [20]:
top_elements_topics

[[('ترکی', 5),
  ('شام', 4),
  ('فوجی کارروائی', 4),
  ('ترک فوجی کارروائی', 3),
  ('آغاز', 2),
  ('سلامتی کونسل', 2),
  ('مطالبہ', 2),
  ('ترکی شام', 1),
  ('کرد جنگجوؤں کیخلاف فوجی کارروائی', 1),
  ('دےیورپی یونین', 1)],
 [('ڈینگی', 7),
  ('ڈی', 6),
  ('مریضوں', 2),
  ('مبتلا مریضوں', 2),
  ('عیادت', 2),
  ('آزادکشمیر', 2),
  ('سی لاہور', 2),
  ('سی خانیوال', 2),
  ('کنگ عبدالله ہسپتال', 1),
  ('بیڈ', 1)],
 [('ڈینگی', 22),
  ('قابو', 4),
  ('انعقاد', 4),
  ('ڈینگی مچھر', 3),
  ('بچاؤ', 3),
  ('بچائو', 2),
  ('لئے', 2),
  ('بچاؤ مہم', 2),
  ('شہری حصہ', 2),
  ('شہباز شاہین', 2)],
 [('سی پیک اتھارٹی', 11),
  ('سی پیک', 6),
  ('حکام', 6),
  ('غربت', 3),
  ('صنعتی تعاون', 3),
  ('دوسرے مرحلہ', 2),
  ('دوسرے مرحلے', 2),
  ('لئے', 2),
  ('چین پاکستان', 2),
  ('مدد', 2)],
 [('سی پیک', 15),
  ('پاکستان', 8),
  ('چین', 7),
  ('دورہ چین', 4),
  ('سی آر', 4),
  ('وفاقی حکومت', 3),
  ('وفاقی وزیر', 2),
  ('وزیر اعظم عمران خان', 2),
  ('وزیراعظم عمران خان', 2),
  ('لئے', 2)],
 [('ڈینگی', 5),
 

In [21]:
# initializing list of lists 
test_list = top_elements_topics
single_list = []
for x in test_list:
    for u,v in x:
        single_list.append(u)
freq = collections.Counter(single_list)


In [22]:
freq

Counter({'ترکی': 1,
         'شام': 1,
         'فوجی کارروائی': 1,
         'ترک فوجی کارروائی': 1,
         'آغاز': 1,
         'سلامتی کونسل': 1,
         'مطالبہ': 1,
         'ترکی شام': 1,
         'کرد جنگجوؤں کیخلاف فوجی کارروائی': 1,
         'دےیورپی یونین': 1,
         'ڈینگی': 8,
         'ڈی': 1,
         'مریضوں': 2,
         'مبتلا مریضوں': 1,
         'عیادت': 2,
         'آزادکشمیر': 1,
         'سی لاہور': 1,
         'سی خانیوال': 1,
         'کنگ عبدالله ہسپتال': 1,
         'بیڈ': 1,
         'قابو': 1,
         'انعقاد': 3,
         'ڈینگی مچھر': 1,
         'بچاؤ': 2,
         'بچائو': 1,
         'لئے': 4,
         'بچاؤ مہم': 1,
         'شہری حصہ': 1,
         'شہباز شاہین': 1,
         'سی پیک اتھارٹی': 1,
         'سی پیک': 2,
         'حکام': 1,
         'غربت': 1,
         'صنعتی تعاون': 1,
         'دوسرے مرحلہ': 1,
         'دوسرے مرحلے': 1,
         'چین پاکستان': 1,
         'مدد': 1,
         'پاکستان': 2,
         'چین': 1,
         'دورہ چین': 1,


In [23]:
common_element=[]
for u,v in freq.items():
    if v>1:
        common_element.append(u)
common_element

['ڈینگی',
 'مریضوں',
 'عیادت',
 'انعقاد',
 'بچاؤ',
 'لئے',
 'سی پیک',
 'پاکستان',
 'ضلعی انتظامیہ',
 'ساتھ اظہار یکجہتی',
 'حوالے']

In [24]:
new_top_elements_topics = []
for top10_in_topic in top_elements_topics:
    new_list_topic = []
    for u,v in top10_in_topic:
        if u not in common_element:
            new_list_topic.append((u,v))
    new_top_elements_topics.append(new_list_topic)

In [26]:
new_top_elements_topics[0]

[('ترکی', 5),
 ('شام', 4),
 ('فوجی کارروائی', 4),
 ('ترک فوجی کارروائی', 3),
 ('آغاز', 2),
 ('سلامتی کونسل', 2),
 ('مطالبہ', 2),
 ('ترکی شام', 1),
 ('کرد جنگجوؤں کیخلاف فوجی کارروائی', 1),
 ('دےیورپی یونین', 1)]

In [35]:
return_clusters_title_sgram(0,dataset,'km_label')

ترکی کی شام میں فوجی کارروائی کا آغاز
شام_ترکی ترکی_کی کارروائی_شام شام_میں کارروائی_فوجی کارروائی_کا کارروائی_آغاز
ترکی نے شام میں فوجی کارروائی کا آغاز کر دیا
کر_ترکی ترکی_نے کر_شام شام_میں کارروائی_فوجی آغاز_کارروائی کارروائی_کا کر_آغاز کر_دیا
ترکی شام میں کرد جنگجوؤں کیخلاف فوجی کارروائی روک دے‘یورپی یونین
یورپی یونین شام میں مجوزہ ’’سیف زون‘‘ کے قیام کے لیے کوئی رقم نہیں دے گی
شام_ترکی دے_شام شام_میں جنگجوؤں_کرد دے_جنگجوؤں جنگجوؤں_کیخلاف کارروائی_فوجی دے_کارروائی یونین_روک یونین_دے‘یورپی یونین_یونین یونین_
 یونین_یورپی دے_یونین دے_شام شام_میں زون‘‘_مجوزہ زون‘‘_’’سیف قیام_زون‘‘ زون‘‘_کے دے_قیام قیام_کے قیام_لیے رقم_کوئی دے_رقم دے_نہیں دے_گی
شام میں ترک فوجی کارروائی پر عالمی برادری چلا اٹھی
صدر سلامتی کونسل کا ترکی سے شہریوں کی جان ومال کا حفاظت یقینی بنانے اور تحمل کا مظاہرہ کرنے کا مطالبہ


ValueError: [E167] Unknown morphological feature: 'ConjType' (9141427322507498425). This can happen if the tagger was trained with a different set of morphological features. If you're using a pretrained model, make sure that your models are up to date:
python -m spacy validate

In [64]:
top_elements_topics[12]

[('ڈینگی', 5),
 ('حوالے', 3),
 ('زیر صدارت انسداد ڈینگی', 2),
 ('اہم اجلاس', 2),
 ('انعقاد', 2),
 ('صورتحال', 2),
 ('اقدامات', 2),
 ('جائزہ', 2),
 ('وجہ', 2),
 ('صورتحال قابو', 2)]

In [65]:
labels_proposed=[]
for i in range(0,kn.knee):
    labels_proposed.append(return_clusters_title(i,dataset,'km_label'))

                      term      rank  wCount
238           شام میں فوجی  0.399441       3
423      میں فوجی کارروائی  0.399441       3
239  شام میں فوجی کارروائی  0.399441       4
334       فوجی کارروائی کا  0.356475       3
569        کارروائی کا غاز  0.356475       3
                    term      rank  wCount
953          ڈی سی لاہور  0.333053       3
751         میں ڈینگی کے  0.274766       3
401        سی لاہور اصغر  0.198980       3
572     لاہور اصغر جوئیہ  0.198980       3
402  سی لاہور اصغر جوئیہ  0.198980       4
               term      rank  wCount
2755  ڈینگی سے بچاؤ  0.437364       3
516    بچاؤ مہم میں  0.321283       3
819    حصہ لے شہباز  0.321283       3
1374    سے بچاؤ مہم  0.321283       3
1527    شہری حصہ لے  0.321283       3
                     term      rank  wCount
433        سی پیک اتھارٹی  0.609845       3
238           حکام سی پیک  0.372439       3
239   حکام سی پیک اتھارٹی  0.372439       4
1074       پیک اتھارٹی کے  0.239567       3
457     سی پیک اتھارٹی ک

In [66]:
all_labels = pd.DataFrame({'Cluster':range(0,kn.knee),'Zero-Order': labels_zero_order,'M-Order':labels_m_order,'T-Order':labels_t_order,'ProposedMethod':labels_proposed})

In [67]:
all_labels

,Cluster,Zero-Order,M-Order,T-Order,ProposedMethod
0,0,ترکی شام,ترک فوجی کارروائی,ترکی شام,ترکی کی شام میں فوجی کارروائی
1,1,ڈی ایچ کیو ہسپتال وہاڑی کادورہ ہسپتال,ڈی ایچ کیو ہسپتال وہاڑی کادورہ ہسپتال,ڈی ایچ کیو ہسپتال وہاڑی کادورہ ہسپتال,ڈینگی کے مریضوں
2,2,ڈینگی مچھر,سی ای او ہیلتھ خانیوال,سی ای او ہیلتھ خانیوال,انعقاد احتیاطی تدابیر اپنا کر ڈینگی پر قابو
3,3,سی پیک اتھارٹی بارے,حکام,حکام,حکام سی پیک اتھارٹی
4,4,وزیراعظم عمران خان,وزیراعظم عمران خان,وزیر اعظم عمران خان,پاکستان اور چین سی پیک
5,5,ضلعی انتظامیہ لاہور,ضلعی انتظامیہ لاہور,ضلعی انتظامیہ لاہور,ضلعی انتظامیہ نے ڈینگی کی رپورٹ
6,6,پریشان عمران خان,پی ٹی آئی حکومت عمران خان,پی ٹی آئی حکومت عمران خان,No title constructed
7,7,کشمیری عوام کیساتھ اظہار یکجہتی,طاقت کشمیری حریت پسندوں,کشمیری عوام کیساتھ اظہار یکجہتی,کشمیریوں سے اظہار یکجہتی کیلئے ریلی
8,8,پاک سری لنکن ٹی ٹونٹی کرکٹ سیریز سکیورٹی,سری لنکن,سری لنکا,No title constructed
9,9,محکمہ صحت ڈینگی وائرس,محکمہ صحت ڈینگی وائرس,محکمہ صحت ڈینگی وائرس,محکمہ صحت ڈینگی وائرس


In [68]:
all_labels.to_csv('all_labels_comparison.csv',sep='\t',encoding='utf-8',index=False)